## Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras import applications, optimizers

## Import image from the directory

In [ ]:
train_dir='../input/100-bird-species/birds/train'
val_dir='../input/100-bird-species/birds/valid'
test_dir='../input/100-bird-species/birds/test'

In [ ]:
train_IDG=ImageDataGenerator(rescale=1/255)
val_IDG=ImageDataGenerator(rescale=1/255)
test_IDG=ImageDataGenerator(rescale=1/255)

In [ ]:
train_img=train_IDG.flow_from_directory(train_dir, target_size=(224,224),
                                              color_mode='rgb', class_mode='sparse',batch_size=256)
val_img=val_IDG.flow_from_directory(val_dir, target_size=(224,224),
                                          color_mode='rgb', class_mode='sparse',batch_size=256)
test_img=test_IDG.flow_from_directory(test_dir, target_size=(224,224),
                                            color_mode='rgb', class_mode='sparse',batch_size=256)

In [ ]:
dirs = train_img.class_indices
rdirs = {v:k  for k,v in dirs.items()}

## Build model with the VGG16

In [ ]:
model = applications.VGG16()
model.summary()

In [ ]:
vgg16 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg16.layers: layer.trainable=False
model=Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(275,activation='softmax'))
print(model.summary())

## Train Model

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_img,validation_data=val_img, epochs=10,verbose=1)

## Evaluate

In [ ]:
model.evaluate(test_img)

In [ ]:
for i in range(5):
    choice = random.randint(0,274)
    dir = rdirs[choice]
    files = os.listdir(test_dir+'/'+dir)
    img = random.choice(files)
    a = cv2.imread(test_dir+'/'+dir+'/'+img)  
    a = cv2.resize(a,(224,224))
    plt.imshow(a)
    plt.show()
    a = np.array(a/255)
    answer = model.predict(np.array([a]))
    pred = np.argmax(answer)
    print("predict:",rdirs[pred],"actual:",rdirs[choice])